In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
import numpy as np
import math as m
from scipy.integrate import quad
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import matplotlib as mpl

mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'STIXGeneral'

mpl.rc('font', size=12)
mpl.rc('axes', titlesize=10) #fontsize of the title
mpl.rc('axes', labelsize=10) #fontsize of the x and y labels
mpl.rc('xtick', labelsize=10) #fontsize of the x tick labels
mpl.rc('ytick', labelsize=10) #fontsize of the y tick labels
mpl.rc('legend', fontsize=10) #fontsize of the legend

b = 1.5
def area_sphere(d):
	return(2*np.pi**(d/2)/m.gamma(d/2.0))

def Cu(d, v):
    return((2.0/(area_sphere(d)*(d-1)))*pow(v,-d-2.0/3))

def Cs(d, v):
    return(2.0/(area_sphere(d))*pow(v,-d-2.0/3))
    
def dvol_RG(d, v, z):
    return area_sphere(d-1) * v**(d-1)* pow(1-z**2,(d-3)/2)

def dvol_flux(d, v):
    return  area_sphere(d-1)*area_sphere(d)*v**(d-1)



# Hydro RG

In [ ]:
b = 1.5

def compute_nu1(d):

    def RG_CH1(d,v,z):
        w = np.sqrt(1 + v**2 - 2*v*z)
        denr = v**(2/3) + w**(2/3)
        factor = (1-z**2)*(2*v*z-1)/w**2
        F1 = factor*v*(v-2*z)*Cu(d,w)
        F2 = factor*(1-v**2)*Cu(d,v)
        return -dvol_RG(d,v,z)*(F1+F2)/denr
      #  return -dvol_RG(d,v,z)*(2*F1)/denr # int F2 = int F1

    def fv_RG_CH1(d,v):
        lower_lim_z = (v**2+1-b**2)/(2*v)
        upper_lim_z = v/2
        return quad(lambda z: RG_CH1(d,v,z), lower_lim_z, upper_lim_z)[0]

    integ_CH1 = quad(lambda v: fv_RG_CH1(d,v), 1, b)[0]
    return  np.sqrt(integ_CH1/(1-b**(-4/3)))


tollerance = 1e-5
maxiter = 150
def compute_nu2(d, nu2_left, nu2_right):
    def RG_CH2(d,nu1,nu2,v,z):
        w = np.sqrt(1 + v**2 - 2*v*z)
        F3_1 = ((1-z**2)*Cu(d,v))/(nu1*v**(2/3) + nu2*w**(2/3))
        F3_2 = ((1-z**2)*(v/w)**2*Cu(d,w))/(nu2*v**(2/3) + nu1*w**(2/3))
        return dvol_RG(d,v,z) * (F3_1+F3_2)
       # return dvol_RG(d,v,z) * (2*F3_2) # int F3_1 = int F3_2

    def fv_RG_CH2(d,nu1,nu2,v):
        lower_lim = (v**2+1-b**2)/(2*v)
        upper_lim = v/2
        return quad(lambda z: RG_CH2(d,nu1,nu2,v,z),lower_lim,upper_lim)[0]

    # Aim to get zero of f(nu2)
    def solve_nu2(nu1, nu2):
        integ_CH2 = quad(lambda v: fv_RG_CH2(d,nu1,nu2,v), 1, b)[0]
        return [nu2*(1-b**(-4/3)) - integ_CH2]

    nu1 = compute_nu1(d)

    x = np.zeros(maxiter); x[0] = nu2_left; x[1] =nu2_right
    for i in range(2,maxiter):
        mid = (x[i-2] + x[i-1])/2
        #print("i , mid ", i , mid)
        if (abs(mid-x[i-1]) < tollerance):
            x[i] = mid
            break
        if solve_nu2(nu1,x[i-2])[0]*solve_nu2(nu1,mid)[0] > 0:
            x[i] = x[i-1]; x[i-1] = mid
        else:
            x[i-1] = x[i-2]; x[i] = mid

    if (i > maxiter):
        print("ERROR: compute_nu1() did not converge!")
    else:
        return mid

for d in range(2,5):
    print("nu1 = ", d, compute_nu1(d))
    print("nu2 = ", d, compute_nu2(d, 0.1,0.5))

# HDT ET 
Kolmogorov's constant. Original Su1u1 & Su2u2 have (1-z^2) that has been removed here. (1-z^2) is adjusted with (1-z^2)^{(d-3)/2}


In [ ]:
from scipy.special import roots_jacobi

def Su1u1(d,nu1,v,z):
    w = np.sqrt(1 + v**2 - 2*v*z)
    denr = nu1*(1 + v**(2/3) + w**(2/3))
    factor = 2*v*z/w**2
    numr1 =  (2*v*z-1)*Cu(d,v)*Cu(d,w)
    numr2 =  (v-2*z)*v*Cu(d,1)*Cu(d,w)
    numr3 = (1-v**2)*Cu(d,1)*Cu(d,v)
    return factor*(numr1+numr2+numr3)/denr

def Su2u2(d,nu1,nu2,v,z):
    w = np.sqrt(1 + v**2 - 2*v*z)
    denr = nu2*(1 + v**(2/3)) + nu1*w**(2/3)
    numr = (v/w)**2*Cu(d,w)*(Cu(d,v)-Cu(d,1))
    return numr/denr

    
def compute_Ko(d, v_lower_lim, nu1_eq_nu2):

    n=100
    alpha =  beta = (d-1)/2
    [rt, wt] = roots_jacobi(n, alpha, beta, mu=False)

    def flux_CH1(d,nu1,v):
        int_z = sum(Su1u1(d,nu1,v,rt)*wt)
        return int_z * dvol_flux(d,v)* np.log(1/v)

    def flux_CH2(d,nu1,nu2,v):
        int_z = sum(Su2u2(d,nu1,nu2,v,rt)*wt)
        return int_z * dvol_flux(d,v) * np.log(1/v)

    if (d==2):
        nu1 = compute_nu1(d)
        integ_flux_CH1 = romberg(lambda v: flux_CH1(d,nu1,v), v_lower_lim, 1)
        return (abs(integ_flux_CH1))**(-2/3)
    else:   # d > 2
        nu2 = compute_nu2(d, 0.3, 0.8)
        if (nu1_eq_nu2):
            nu1 = nu2
        else:
            nu1 = compute_nu1(d)

        integ_flux_CH1 = quad(lambda v: flux_CH1(d,nu1,v), v_lower_lim, 1)[0]
        integ_flux_CH2 = quad(lambda v: flux_CH2(d,nu1,nu2,v), 1e-7, 1)[0]
        return (integ_flux_CH1+(d-2)*integ_flux_CH2)**(-2/3)

print(compute_Ko(3, 0.22, False))

# MHD

In [10]:
def compute_eta1(d):

    def RG_CH1(d,v,z):
        w = np.sqrt(1 + v**2 - 2*v*z)
        denr = v**(2/3) + w**(2/3)
        F1 = (v/w)**2* (1-z**2)*z**2* Cu(d,w)
        F2 = (1-z**2)*(1-v*z)*(v-z)*Cu(d,v)/w**2
        return dvol_RG(d,v,z)*(F1+F2)/denr

    def fv_RG_CH1(d,v):
        lower_lim_z = (v**2+1-b**2)/(2*v)
        upper_lim_z = v/2
        return quad(lambda z: RG_CH1(d,v,z), lower_lim_z, upper_lim_z)[0]

    integ_CH1 = quad(lambda v: fv_RG_CH1(d,v), 1, b)[0]
    return np.sqrt(integ_CH1/(1-b**(-4/3)))

tollerance = 1e-5
maxiter = 150
def compute_eta2(d, eta2_left, eta2_right):
    def RG_CH2(d,eta1,eta2,v,z):
        w = np.sqrt(1 + v**2 - 2*v*z)
        F3_1 = ((1-z**2)*Cu(d,v))/(eta1*v**(2/3) + eta2*w**(2/3))
        F3_2 = ((1-z**2)*(v/w)**2*Cu(d,w))/(eta2*v**(2/3) + eta1*w**(2/3))
        return dvol_RG(d,v,z) * (F3_1+F3_2)

    def fv_RG_CH2(d,eta1,eta2,v):
        lower_lim = (v**2+1-b**2)/(2*v)
        upper_lim = v/2
        return quad(lambda z: RG_CH2(d,eta1,eta2,v,z),lower_lim,upper_lim)[0]

    # Aim to get zero of f(nu2)
    def solve_eta2(eta1, eta2):
        integ_CH2 = quad(lambda v: fv_RG_CH2(d,eta1,eta2,v), 1, b)[0]
        return [eta2*(1-b**(-4/3)) - integ_CH2]

    eta1 = compute_eta1(d)

    x = np.zeros(maxiter); x[0] = eta2_left; x[1] =eta2_right
    for i in range(2,maxiter):
        mid = (x[i-2] + x[i-1])/2
        if (abs(mid-x[i-1]) < tollerance):
            x[i] = mid
            break
        if solve_eta2(eta1,x[i-2])[0]*solve_eta2(eta1,mid)[0] > 0:
            x[i] = x[i-1]; x[i-1] = mid
        else:
            x[i-1] = x[i-2]; x[i] = mid

    if (i > maxiter):
        print("ERROR: compute_nu1() did not converge!")
    else:
        return mid

for d in range(2,6):
    print("eta1 = ", d, compute_eta1(d))
    print("eta2 = ", d, compute_eta2(d, 0.1,0.5))

eta1 =  2 0.24755140493062633
eta2 =  2 0.499993896484375
eta1 =  3 0.21407979698505752
eta2 =  3 0.472198486328125
eta1 =  4 0.19271841920834573
eta2 =  4 0.4180236816406251
eta1 =  5 0.17722151427570576
eta2 =  5 0.37855834960937507


# eta estimate

In [ ]:
b = 1.5
d = 2
Ad = 2*area_sphere(d-1)/(area_sphere(d)*(d-1))
nu2 = (3/4)*np.sqrt(Ad*3/8)
factor = (3/4)* np.sqrt(2*Ad*(b-1))* (3/4)**((d-1)/4)

asymp_eta1 = factor*np.sqrt(3/8)
asymp_eta2 = factor
print("d, eta1_2_estimate = ", d, nu2, asymp_eta1, asymp_eta2)

# MHD ET
Kolmogorov's constant. Original Su1u1 & Su2u2 have (1-z^2) that has been removed here. (1-z^2) is adjusted with (1-z^2)^{(d-3)/2}

In [7]:
from scipy.special import roots_jacobi

def Su1u1(d,eta1,v,z):
    w = np.sqrt(1 + v**2 - 2*v*z)
    denr = eta1*(1 + v**(2/3)+w**(2/3))
    numr = (v/w)**2* z**2* (Cu(d,v)-Cu(d,1))* Cu(d,w)
    return numr/denr

def Su2u2(d,eta1,eta2,v,z):
    w = np.sqrt(1 + v**2 - 2*v*z)
    denr = eta2*(1 + v**(2/3)) + eta1*w**(2/3)
    numr = (v/w)**2* (Cu(d,v)-Cu(d,1))* Cu(d,w)
    return numr/denr


def compute_Ko(d, v_lower_lim, eta1_eq_eta2):

    n=100
    alpha =  beta = (d-1)/2
    [rt, wt] = roots_jacobi(n, alpha, beta, mu=False)

    def flux_CH1(d,eta1,v):
        int_z = sum(Su1u1(d,eta1,v,rt)*wt)
        return int_z * dvol_flux(d,v)* np.log(1/v)

    def flux_CH2(d,eta1,eta2,v):
        int_z = sum(Su2u2(d,eta1,eta2,v,rt)*wt)
        return int_z * dvol_flux(d,v) * np.log(1/v)

    if (d==2):
        eta1 = compute_eta1(d)
        print("eta1 = ",eta1)
        integ_flux_CH1 = quad(lambda v: flux_CH1(d,eta1,v), v_lower_lim, 1)[0]
        return ((integ_flux_CH1))**(-2/3)
    else:   # d > 2
        eta1 = compute_eta1(d)
        eta2 = compute_eta2(d, 0.3, 0.8)
        integ_flux_CH1 = quad(lambda v: flux_CH1(d,eta1,v), v_lower_lim, 1)[0]
        integ_flux_CH2 = quad(lambda v: flux_CH2(d,eta1,eta2,v), 1e-7, 1)[0]
        return (integ_flux_CH1+(d-2)*integ_flux_CH2)**(-2/3)

#v_lower_lim = 1e-7
#Ko = compute_Ko(d, v_lower_lim, True)
#print("nu1 = nu2: Ko = ", Ko)

v_lower_lim = 1e-6
for d in range(2,6):
    Ko = compute_Ko(d, v_lower_lim, False)
    print("d, Ko = ", d, Ko)


eta1 =  0.24755140493062633
d, Ko =  2 0.9262816970550424
d, Ko =  3 0.9992972873290293
d, Ko =  4 1.0718263254487965
d, Ko =  5 1.1362838660008503
